In [1]:
pip install natasha

  Obtaining dependency information for natasha from https://files.pythonhosted.org/packages/32/9c/bb9d33c13564bcc939bb727087ef51b16ed3b49cc3b8fdec07c87b02f1de/natasha-1.6.0-py3-none-any.whl.metadata
  Using cached natasha-1.6.0-py3-none-any.whl.metadata (23 kB)
  Obtaining dependency information for pymorphy2 from https://files.pythonhosted.org/packages/07/57/b2ff2fae3376d4f3c697b9886b64a54b476e1a332c67eee9f88e7f1ae8c9/pymorphy2-0.9.1-py3-none-any.whl.metadata
  Using cached pymorphy2-0.9.1-py3-none-any.whl.metadata (3.6 kB)
  Obtaining dependency information for razdel>=0.5.0 from https://files.pythonhosted.org/packages/15/2c/664223a3924aa6e70479f7d37220b3a658765b9cfe760b4af7ffdc50d38f/razdel-0.5.0-py3-none-any.whl.metadata
  Using cached razdel-0.5.0-py3-none-any.whl.metadata (10.0 kB)
  Obtaining dependency information for navec>=0.9.0 from https://files.pythonhosted.org/packages/bc/c1/771ec5565f0ce24874d7fd325b429f9caa80517a40d2e4ce5705120591f3/navec-0.10.0-py3-none-any.whl.metadat

In [3]:
pip install rapidfuzz

  Obtaining dependency information for rapidfuzz from https://files.pythonhosted.org/packages/c9/5a/d00e1f63564050a20279015acb29ecaf41646adfacc6ce2e1e450f7f2633/rapidfuzz-3.13.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
    --------------------------------------- 0.0/1.6 MB 660.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.6 MB 1.3 MB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.6 MB 2.2 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.6 MB 2.2 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.6 MB 3.1 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.6 MB 2.7 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.6 MB 3.3 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.5 MB/s eta 0:00:00
Note: you may need to restart th

In [117]:
import requests
import pandas as pd
from pymorphy2 import MorphAnalyzer
from rapidfuzz import fuzz

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
morph = MorphAnalyzer()

def fetch_moex_tickers():
    url = 'https://iss.moex.com/iss/engines/stock/markets/shares/securities.json'
    response = requests.get(url)
    data = response.json()
    securities = data['securities']['data']
    columns = data['securities']['columns']
    df = pd.DataFrame(securities, columns=columns)
    df_filtered = df[['SECID', 'SHORTNAME', 'SECNAME']].dropna()
    company_to_ticker = {}
    for _, row in df_filtered.iterrows():
        names = [row['SECNAME'], row['SHORTNAME']]
        for name in names:
            company_to_ticker[name.lower()] = row['SECID']
    return company_to_ticker

def normalize_text(text):
    return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])

def extract_company_names(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    orgs_normalized = [span.normal for span in doc.spans if span.type == 'ORG']
    orgs_original = [span.text for span in doc.spans if span.type == 'ORG']
    
    # Удаляем шумовые фразы
    stopwords = {"сд", "совет директоров", "акционеров", "мсфо"} 
    result = []
    for original, normalized in zip(orgs_original, orgs_normalized):
        norm_text = normalized.lower()
        if norm_text not in stopwords:
            print(f"Original: {original} -> Normalized: {normalized}")
            result.append(norm_text)
    return result

def get_ticker_from_text(text, company_to_ticker):
    orgs = extract_company_names(text)
    for org in orgs:
        org_norm = normalize_text(org)
        best_match = None
        best_score = 0
        for key, ticker in company_to_ticker.items():
            score = fuzz.partial_ratio(org_norm, key)
            if score > best_score and score > 80: 
                best_match = (key, ticker)
                best_score = score
        if best_match:
            return best_match
    return None, None

# Example usage
if __name__ == "__main__":
    sample_text = """
Газпром что-то там вообще такое
"""
    company_to_ticker = fetch_moex_tickers()
    company_name, ticker = get_ticker_from_text(sample_text, company_to_ticker)
    if ticker:
        print(f"✅ Extracted Company Name: {company_name}")
        print(f"✅ Extracted Ticker: {ticker}")
    else:
        print("❌ No matching company or ticker found.")


Original: Газпром -> Normalized: Газпром
✅ Extracted Company Name: "газпром" (пао) ао
✅ Extracted Ticker: GAZP


In [106]:
file_path = r"C:\Users\Карпенко\ВКР Карпенко\data makeup\filtered_interfax_data.xlsx"

# Загружаем Excel-файл
df = pd.read_excel(file_path)

# Фильтруем строки, где Label=1
filtered_df = df[df['Label'] == 1]

# Выводим первые строки для проверки
filtered_df

,Дата,Текст,Label
6,2025-01-30 13:09:25,"🍷**EBITDA LTM ""Новабев групп"" на 30 июня 2024 ...",1.0
8,2025-01-30 11:39:26,"🔺**""Соллерс"" поднял пороговое значение учитыва...",1.0
9,2025-01-30 11:27:51,"🗓**""НОВАТЭК"" 6 февраля проведет сбор заявок на...",1.0
10,2025-01-30 11:02:33,"**Минфин не слышал о планах ""Транснефти"" снизи...",1.0
13,2025-01-30 08:02:19,**💰SoftBank Group ведет переговоры об инвестиц...,1.0
...,...,...,...
1504,2025-04-01 08:21:01,**Роснано допустило техдефолт по купону облига...,1.0
1508,2025-04-01 07:14:17,"🗓**Совет директоров ""Новабев Групп"" 2 апреля р...",1.0
1512,2025-04-01 07:03:16,"**""Циан"" одновременно со стартом торгов на Мос...",1.0
1513,2025-04-01 07:00:48,"**""Циан"" в IV квартале нарастил выручку на 5%,...",1.0


In [ ]:
correctness = []

for index, row in filtered_df.iterrows():
    print(f"\n📰 News:\n{row['Текст']}")
    print(f"🔎 Extracted Ticker: {row['Ticker']}")
    
    user_input = input("0 or 1")
    
    if user_input not in ['0', '1']:
        print("Error")
        user_input = '0'
    
    correctness.append(int(user_input))

df_filtered['Correct'] = correctness

# Сохраняем результат
output_path = r"C:\Users\Карпенко\ВКР Карпенко\model\filtered_with_tickers_corrected.xlsx"
df_filtered.to_excel(output_path, index=False)

##  error correction

In [10]:
# Подсчет точности
total = len(df_filtered)
correct = df_filtered['Correct'].sum()
accuracy = correct / total

print(f"📊 Accuracy (точность): {accuracy:.4f} ({correct}/{total})")

📊 Accuracy (точность): 0.7018 (266/379)


In [57]:
df_filtered = pd.read_excel(r"C:\Users\Карпенко\ВКР Карпенко\model\filtered_with_tickers_corrected.xlsx")
df_filtered

,Дата,Текст,Label,Ticker,Correct
0,2025-01-30 13:09:25,"🍷**EBITDA LTM ""Новабев групп"" на 30 июня 2024 ...",1,BELU,1
1,2025-01-30 11:39:26,"🔺**""Соллерс"" поднял пороговое значение учитыва...",1,SVAV,1
2,2025-01-30 11:27:51,"🗓**""НОВАТЭК"" 6 февраля проведет сбор заявок на...",1,NVTK,1
3,2025-01-30 11:02:33,"**Минфин не слышал о планах ""Транснефти"" снизи...",1,TRNFP,1
4,2025-01-30 08:02:19,**💰SoftBank Group ведет переговоры об инвестиц...,1,BSPB,0
...,...,...,...,...,...
374,2025-04-01 08:21:01,**Роснано допустило техдефолт по купону облига...,1,Not found,0
375,2025-04-01 07:14:17,"🗓**Совет директоров ""Новабев Групп"" 2 апреля р...",1,BELU,1
376,2025-04-01 07:03:16,"**""Циан"" одновременно со стартом торгов на Мос...",1,CNRU,1
377,2025-04-01 07:00:48,"**""Циан"" в IV квартале нарастил выручку на 5%,...",1,CNRU,1


## error correction

In [171]:
import re
import requests
import pandas as pd
from natasha import Segmenter, NewsNERTagger, NewsEmbedding, MorphVocab, Doc
from pymorphy2 import MorphAnalyzer
from rapidfuzz import fuzz

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
morph = MorphAnalyzer()

def clean_company_name(name):
    keywords_to_remove = ['plc', 'ipjsc', 'оао', 'зао', 'пиф', 'international public js com', 'мкпао',
                         'адр', 'corporate center', 'группа', 'пао']
    words = name.lower().split()
    cleaned = ' '.join([w for w in words if w not in keywords_to_remove])
    return cleaned


def fetch_moex_tickers():
    url = 'https://iss.moex.com/iss/engines/stock/markets/shares/securities.json'
    response = requests.get(url)
    data = response.json()
    securities = data['securities']['data']
    columns = data['securities']['columns']
    df = pd.DataFrame(securities, columns=columns)
    
    df_filtered = df[['SECID', 'SHORTNAME', 'SECNAME', 'LATNAME']].dropna()
    df_filtered = df_filtered[df_filtered['SECID'].apply(lambda x: len(str(x)) <= 6)]
    
    company_to_ticker = {}
    for _, row in df_filtered.iterrows():
        names = [row['SECNAME'], row['SHORTNAME'], row['LATNAME']]
        for name in names:
            company_to_ticker[name.lower()] = row['SECID']
    return company_to_ticker


def normalize_text(text):
    return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])

def extract_company_names(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    orgs_normalized = [span.normal for span in doc.spans if span.type == 'ORG']
    orgs_original = [span.text for span in doc.spans if span.type == 'ORG']
    
    stopwords = {"сд", "совет директоров", "акционеров", "мсфо", 'цб',
                "мосбиржа", 'банк россии', 'рбк', 'московская биржа', 'ук'}
    result = []
    for original, normalized in zip(orgs_original, orgs_normalized):
        norm_text = normalized.lower()
        if norm_text not in stopwords:
            print(f"Original: {original} -> Normalized: {normalized}")
            result.append(norm_text)

    if "лента" in text.lower() and not any("лента" in org for org in result):
        result.append("лента")
        
    if "самолет" in text.lower() and not any("самолет" in org for org in result):
        result.append("самолет")

    return result


def replace_sber_with_sberbank(text):
    text = re.sub(r'\bСбер\b(?=\s|$)', 'Сбербанк', text)
    text = re.sub(r'\bТ[ -]Банк\b', 'Т-Технологии', text, flags=re.IGNORECASE)
    text = re.sub(r'\bТ[ -]Банка\b', 'Т-Технологии', text, flags=re.IGNORECASE)
    text = re.sub(r'["«]?\s*Лента\s*["»]?', 'Лента АО', text, flags=re.IGNORECASE)
    return text

def get_ticker_from_text(text, company_to_ticker):
    text = replace_sber_with_sberbank(text)
    orgs = extract_company_names(text)
    
    for org in orgs:
        org_norm = normalize_text(org)
        if org_norm.lower() in ["vk", "вк", "вконтакте", "vk tech"]:
            return ("VK", "VKCO")
        if org_norm.lower() in ["x5", "x5 group", "х5", "икс5", "икс 5"]:
            return ("X5", "X5")
        if org_norm.lower() in ["магнит"]:
            return ('ПАО "Магнит"' , "MGNT")
        if org_norm.lower() in ["мать и дитя", "матери и дети", "md medical group"]:
            return ('ПАО "Мать и дитя"', "MDMG")
        if org_norm.lower() in ["cамолет"]:
            return ('Самолет', "SMLT")

        
        org_cleaned = clean_company_name(org_norm)
        best_match = None
        best_score = 0
        for key, ticker in company_to_ticker.items():
            key_cleaned = clean_company_name(key)
            score = fuzz.partial_ratio(org_cleaned, key_cleaned)
            if score > best_score and score > 85:
                best_match = (key, ticker)
                best_score = score
        if best_match:
            return best_match
    
    stopwords = {"сд", "совет директоров", "акционеров", "мсфо", 'цб',
                 'мосбиржа', 'банк россии', 'рбк', 'московская биржа'}
    for org in orgs:
        if org.lower() not in stopwords:
            return (org, "N/A")
    
    if "ооо" in text.lower():
        return ("ООО", "N/A")
    elif any(keyword in text.lower() for keyword in ["plc", "ltd", "limited", "inc", "corp", "company"]):
        return ("Иностранная компания", "N/A")
    else:
        return (None, None)



if __name__ == "__main__":
    sample_text = """
**"Самолет" в 2024 году увеличил EBITDA на 16%**

Девелопер "Самолет" в 2024 году увеличил EBITDA по МСФО на 16%, до 83,6 млрд рублей.

Согласно отчетности, чистая прибыль упала в 3 раза, до 8,2 млрд рублей.

Накануне аналитики ожидали EBITDA "Самолета" в диапазоне 74-78,3 млрд рублей, чистую прибыль - 5-7,1 млрд рублей.

@ifax_go
    """
    company_to_ticker = fetch_moex_tickers()
    company_name, ticker = get_ticker_from_text(sample_text, company_to_ticker)
    if ticker:
        print(f"✅ Extracted Company Name: {company_name}")
        print(f"✅ Extracted Ticker: {ticker}")
    else:
        print("❌ No matching company or ticker found.")

✅ Extracted Company Name: гк самолет ао
✅ Extracted Ticker: SMLT


In [125]:
df = df_filtered[['Текст']].copy()

company_to_ticker = fetch_moex_tickers()

df['Ticker'] = df['Текст'].apply(lambda x: get_ticker_from_text(x, company_to_ticker)[1])

output_path = r"C:\Users\Карпенко\ВКР Карпенко\data markup\filtered_with_tickers_2.xlsx"
df.to_excel(output_path, index=False)

print(f" {output_path}")

Original: Новабев групп -> Normalized: Новабев групп
Original: Новабев Групп -> Normalized: Новабев Групп
Original: LTM -> Normalized: LTM
Original: LTM -> Normalized: LTM
Original: Соллерс -> Normalized: Соллерс
Original: ПАО "Соллерс" -> Normalized: ПАО "Соллерс"
Original: Соллерса -> Normalized: Соллерса
Original: Соллерс -> Normalized: Соллерс
Original: НОВАТЭК -> Normalized: НОВАТЭК
Original: ПАО "НОВАТЭК" -> Normalized: ПАО "НОВАТЭК"
Original: Интерфаксу -> Normalized: Интерфаксу
Original: Банка России -> Normalized: Банка России
Original: Минфин -> Normalized: Минфин
Original: Транснефти -> Normalized: Транснефти
Original: Министерство финансов -> Normalized: Министерство финансов
Original: Транснефти -> Normalized: Транснефти
Original: ПАО "Транснефть" -> Normalized: ПАО "Транснефть"
Original: Транснефти -> Normalized: Транснефти
Original: Транснефть -> Normalized: Транснефть
Original: SoftBank Group -> Normalized: SoftBank Group
Original: Financial Times -> Normalized: Financi

Original: Taiwan Semiconductor Manufacturing Company -> Normalized: Taiwan Semiconductor Manufacturing Company
Original: CNBC -> Normalized: CNBC
Original: TSMC -> Normalized: TSMC
Original: Taiex -> Normalized: Taiex
Original: Bank of New York Mellon -> Normalized: Bank of New York Mellon
Original: МКАО "Скилбокс Холдинг -> Normalized: МКАО "Скилбокс Холдинг
Original: МКАО -> Normalized: МКАО
Original: Скилбокс Холдинг -> Normalized: Скилбокс Холдинг
Original: VK -> Normalized: VK
Original: Mail.ru Group -> Normalized: Mail.ru Group
Original: Skillbox -> Normalized: Skillbox
Original: GeekBrains -> Normalized: GeekBrains
Original: Skillbox Limited -> Normalized: Skillbox Limited
Original: Skillbox -> Normalized: Skillbox
Original: IT-предпринимателя -> Normalized: IT-предпринимателя
Original: Almaz Capital -> Normalized: Almaz Capital
Original: ЕГРЮЛ -> Normalized: ЕГРЮЛ
Original: ООО "Скилбокс" -> Normalized: ООО "Скилбокс"
Original: ООО "Гикбреинс" -> Normalized: ООО "Гикбреинс"
Ori

Original: Русснефть -> Normalized: Русснефть
Original: ПАО "Русснефть" -> Normalized: ПАО "Русснефть"
Original: НК -> Normalized: НК
Original: ФосАгро -> Normalized: ФосАгро
Original: ФосАгро -> Normalized: ФосАгро
Original: ФосАгро -> Normalized: ФосАгро
Original: Черкизово -> Normalized: Черкизово
Original: Черкизово -> Normalized: Черкизово
Original: Черкизово -> Normalized: Черкизово
Original: Промсвязьбанк -> Normalized: Промсвязьбанк
Original: ПАО "Промсвязьбанк" -> Normalized: ПАО "Промсвязьбанк"
Original: Интерфаксу -> Normalized: Интерфаксу
Original: ПСБ -> Normalized: ПСБ
Original: АКРА -> Normalized: АКРА
Original: НОВАТЭКа -> Normalized: НОВАТЭКа
Original: НОВАТЭКа -> Normalized: НОВАТЭКа
Original: НОВАТЭКа -> Normalized: НОВАТЭКа
Original: Черкизово -> Normalized: Черкизово
Original: ПАО "Группа Черкизово -> Normalized: ПАО "Группа Черкизово
Original: Норникеля -> Normalized: Норникеля
Original: Норникеля -> Normalized: Норникеля
Original: Сбербанк -> Normalized: Сбербанк


Original: ПАО "ДжетЛенд Холдинг -> Normalized: ПАО "ДжетЛенд Холдинг
Original: СПБ бирже -> Normalized: СПБ бирже
Original: Черкизово -> Normalized: Черкизово
Original: ПАО "Группа Черкизово -> Normalized: ПАО "Группа Черкизово
Original: БК "Регион" -> Normalized: БК "Регион"
Original: Россельхозбанк -> Normalized: Россельхозбанк
Original: Черкизово -> Normalized: Черкизово
Original: КС -> Normalized: КС
Original: ГК "О'Кей -> Normalized: ГК "О'Кей
Original: О'Кей -> Normalized: О'Кей
Original: О'кей Груп -> Normalized: О'кей Груп
Original: МКПАО -> Normalized: МКПАО
Original: ПАО "Элемент" -> Normalized: ПАО "Элемент"
Original: ПАО "Элемент" -> Normalized: ПАО "Элемент"
Original: Элемента -> Normalized: Элемента
Original: Т-Технологии -> Normalized: Т-Технологии
Original: Т-Технологии -> Normalized: Т-Технологии
Original: Росбанк -> Normalized: Росбанк
Original: ИФ -> Normalized: ИФ
Original: Т -> Normalized: Т
Original: Т-Технологий -> Normalized: Т-Технологий
Original: ЛУКОЙЛа -> No

Original: Диасофт -> Normalized: Диасофт
Original: ПАО "Диасофт" -> Normalized: ПАО "Диасофт"
Original: Юнипро -> Normalized: Юнипро
Original: ПАО "Юнипро" -> Normalized: ПАО "Юнипро"
Original: Диасофт -> Normalized: Диасофт
Original: ПАО "Диасофт" -> Normalized: ПАО "Диасофт"
Original: Санкт-Петербург -> Normalized: Санкт-Петербург
Original: Борец -> Normalized: Борец
Original: Национальный расчетный депозитарий (НРД) -> Normalized: Национальный расчетный депозитарий (НРД)
Original: НРД -> Normalized: НРД
Original: Генпрокуратуры -> Normalized: Генпрокуратуры
Original: HeadHunter -> Normalized: HeadHunter
Original: CEO -> Normalized: CEO
Original: HeadHunter (HH) -> Normalized: HeadHunter (HH)
Original: HH -> Normalized: HH
Original: МТС -> Normalized: МТС
Original: ПАО "МТС" -> Normalized: ПАО "МТС"
Original: ПАО "МТС" -> Normalized: ПАО "МТС"
Original: Юрент -> Normalized: Юрент
Original: МТС-банка -> Normalized: МТС-банка
Original: МТС-банка -> Normalized: МТС-банка
Original: Полюс

Original: Абрау-Дюрсо -> Normalized: Абрау-Дюрсо
Original: ГК "Абрау-Дюрсо" -> Normalized: ГК "Абрау-Дюрсо"
Original: Интерфакса -> Normalized: Интерфакса
Original: Софтлайна -> Normalized: Софтлайна
Original: ПАО "Софтлайн" -> Normalized: ПАО "Софтлайн"
Original: ПАО "Софтлайн" -> Normalized: ПАО "Софтлайн"
Original: Совкомбанк -> Normalized: Совкомбанк
Original: Совкомбанк -> Normalized: Совкомбанк
Original: Совкомбанк -> Normalized: Совкомбанк
Original: Набсовет -> Normalized: Набсовет
Original: VK -> Normalized: VK
Original: МКПАО "ВК" -> Normalized: МКПАО "ВК"
Original: VK -> Normalized: VK
Original: VK -> Normalized: VK
Original: Русал -> Normalized: Русал
Original: Русал -> Normalized: Русал
Original: Газпромбанк -> Normalized: Газпромбанк
Original: Эксперт РА -> Normalized: Эксперт РА
Original: Ламбумиз -> Normalized: Ламбумиз
Original: Ламбумиз -> Normalized: Ламбумиз
Original: ИФ -> Normalized: ИФ
Original: HeadHunter -> Normalized: HeadHunter
Original: HeadHunter -> Normaliz

Original: Промомеда -> Normalized: Промомеда
Original: Промомед -> Normalized: Промомед
Original: ПАО "Промомед" -> Normalized: ПАО "Промомед"
Original: Промомеда -> Normalized: Промомеда
Original: ГК "Самолет" -> Normalized: ГК "Самолет"
Original: ГК "Самолет" -> Normalized: ГК "Самолет"
Original: Совкомбанк -> Normalized: Совкомбанк
Original: Газпромбанк -> Normalized: Газпромбанк
Original: ВТБ Капитал трейдинг -> Normalized: ВТБ Капитал трейдинг
Original: АКРА -> Normalized: АКРА
Original: Сбербанка -> Normalized: Сбербанка
Original: Сбербанка -> Normalized: Сбербанка
Original: Набсовет -> Normalized: Набсовет
Original: Сбербанка -> Normalized: Сбербанка
Original: Северсталь -> Normalized: Северсталь
Original: Северсталь -> Normalized: Северсталь
Original: Интерфаксом -> Normalized: Интерфаксом
Original: Северстали -> Normalized: Северстали
Original: Северстали -> Normalized: Северстали
Original: Группы Астра -> Normalized: Группы Астра
Original: ПАО "Группа Астра -> Normalized: ПАО

Original: Сбербанк -> Normalized: Сбербанк
Original: Сбербанк -> Normalized: Сбербанк
Original: Селигдар -> Normalized: Селигдар
Original: АЛРОСА -> Normalized: АЛРОСА
Original: Якутия 24 -> Normalized: Якутия 24
Original: АЛРОСА -> Normalized: АЛРОСА
Original: Селигдар -> Normalized: Селигдар
Original: ПАО "Селигдар" -> Normalized: ПАО "Селигдар"
Original: Газпромбанк -> Normalized: Газпромбанк
Original: ВТБ Капитал трейдинг -> Normalized: ВТБ Капитал трейдинг
Original: Совкомбанк -> Normalized: Совкомбанк
Original: Яндекс -> Normalized: Яндекс
Original: Яндекс -> Normalized: Яндекс
Original: Интерфаксу -> Normalized: Интерфаксу
Original: Банка России -> Normalized: Банка России
Original: АО "Старт капитал -> Normalized: АО "Старт капитал
Original: Московской биржей -> Normalized: Московской биржей
Original: VK -> Normalized: VK
Original: ЗПИФы -> Normalized: ЗПИФы
Original: VK (МКПАО "ВК") -> Normalized: VK (МКПАО "ВК")
Original: VK -> Normalized: VK
Original: VK -> Normalized: VK
Or

## Manual verification of results

In [176]:
df_filtered = pd.read_excel(r"C:\Users\Карпенко\ВКР Карпенко\data markup\filtered_with_tickers_2.xlsx")


total = len(df_filtered)
correct = df_filtered['Correct'].sum()
accuracy = correct / total

print(f"📊 Accuracy (точность): {accuracy:.4f} ({correct}/{total})")

📊 Accuracy (точность): 0.9208 (349/379)
